In [1]:
import ast
import astor

In [2]:

class UnnestFunctionCalls(ast.NodeTransformer):
    def __init__(self):
        self.temp_count = 0

    def generate_temp_name(self):
        name = f"temp_{self.temp_count}"
        self.temp_count += 1
        return name

    def visit_Call(self, node):
        # First, ensure all child nodes are processed
        self.generic_visit(node)

        # Check if there are function calls among the arguments
        for i, arg in enumerate(node.args):
            if isinstance(arg, ast.Call):
                # Generate a new temporary variable for the nested call
                temp_name = self.generate_temp_name()
                temp_var = ast.Name(id=temp_name, ctx=ast.Store())

                # Replace the nested call with the temporary variable
                node.args[i] = ast.Name(id=temp_name, ctx=ast.Load())

                # Create an assignment for the nested call
                assign = ast.Assign(targets=[temp_var], value=arg)

                # Return a block of statements (assignment + current call)
                return ast.Expr(value=ast.Tuple(elts=[assign, node], ctx=ast.Load()))

        return node

def unnest_function_calls(code):
    tree = ast.parse(code, mode='eval')
    transformer = UnnestFunctionCalls()
    new_tree = transformer.visit(tree)
    return astor.to_source(new_tree)


In [3]:

# Example usage
code = "f(g(h(1), 2), i(3, j(4)))"
unnested_code = unnest_function_calls(code)
print(unnested_code)


f(

temp_0 = h(1), g(temp_0, 2), 

temp_1 = j(4), i(3, temp_1))



In [4]:
import ast
import astor
import unittest

class UnnestFunctionCalls(ast.NodeTransformer):
    def __init__(self):
        self.temp_count = 0

    def generate_temp_name(self):
        name = f"temp_{self.temp_count}"
        self.temp_count += 1
        return name

    def visit_Call(self, node):
        # Ensure all child nodes are processed
        self.generic_visit(node)

        # Initialize a block of statements if necessary
        new_stmts = []

        # Replace function calls in arguments with temporary variables
        for i, arg in enumerate(node.args):
            if isinstance(arg, ast.Call):
                temp_name = self.generate_temp_name()
                temp_var = ast.Name(id=temp_name, ctx=ast.Store())

                node.args[i] = ast.Name(id=temp_name, ctx=ast.Load())
                assign = ast.Assign(targets=[temp_var], value=arg)
                new_stmts.append(assign)

        if new_stmts:
            new_stmts.append(ast.Expr(value=node))
            return ast.Tuple(elts=new_stmts, ctx=ast.Load())

        return node

def unnest_function_calls(code):
    tree = ast.parse(code, mode='eval')
    transformer = UnnestFunctionCalls()
    new_tree = transformer.visit(tree)
    return astor.to_source(new_tree).strip()

class TestUnnestFunctionCalls(unittest.TestCase):
    def test_single_nested_call(self):
        code = "f(g(h(1)))"
        expected = "(temp_0 = h(1), temp_1 = g(temp_0), f(temp_1))"
        self.assertEqual(unnest_function_calls(code), expected)

    def test_multiple_arguments(self):
        code = "f(g(h(1), 2), i(3, j(4)))"
        expected = "(temp_0 = h(1), temp_1 = g(temp_0, 2), temp_2 = j(4), temp_3 = i(3, temp_2), f(temp_1, temp_3))"
        self.assertEqual(unnest_function_calls(code), expected)

    def test_no_nested_calls(self):
        code = "f(1, 2)"
        expected = "f(1, 2)"
        self.assertEqual(unnest_function_calls(code), expected)

    def test_nested_calls_with_keywords(self):
        code = "f(x=g(h(1)), y=i(3))"
        expected = "(temp_0 = h(1), temp_1 = g(temp_0), temp_2 = i(3), f(x=temp_1, y=temp_2))"
        self.assertEqual(unnest_function_calls(code), expected)

    def test_nested_in_list(self):
        code = "f([g(h(1)), i(j(2))])"
        expected = "(temp_0 = h(1), temp_1 = g(temp_0), temp_2 = j(2), temp_3 = i(temp_2), f([temp_1, temp_3]))"
        self.assertEqual(unnest_function_calls(code), expected)

    def test_nested_in_dict(self):
        code = "f({'a': g(h(1)), 'b': i(j(2))})"
        expected = "(temp_0 = h(1), temp_1 = g(temp_0), temp_2 = j(2), temp_3 = i(temp_2), f({'a': temp_1, 'b': temp_3}))"
        self.assertEqual(unnest_function_calls(code), expected)

unittest.main()

E
ERROR: /home/fleeb/ (unittest.loader._FailedTest./home/fleeb/)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/fleeb/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/home/fleeb/miniconda3/envs/alphageometry/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:

import ast
import astor


class Unnester(ast.NodeTransformer):
	def __init__(self):
		self.var_counter = 0

	def generate_variable_name(self):
		name = f"temp_{self.var_counter}"
		self.var_counter += 1
		return name

	def visit_Call(self, node):
		# Recursively visit the arguments of the call that might contain other calls
		self.generic_visit(node)

		# Create a temporary variable for each call
		temp_var_name = self.generate_variable_name()
		temp_var = ast.Name(id=temp_var_name, ctx=ast.Store())

		# Replace the original call with a variable name
		new_node = ast.Name(id=temp_var_name, ctx=ast.Load())

		# Create an assignment of the call to the temporary variable
		assign = ast.Assign(targets=[temp_var], value=node)

		# Instead of the original node, return the assignment and the usage of the new variable
		return ast.Expr(value=ast.Tuple(elts=[assign, new_node], ctx=ast.Load()))


def unnest_calls(code):
	tree = ast.parse(code)
	unnester = Unnester()
	new_tree = unnester.visit(tree)
	new_tree = ast.fix_missing_locations(new_tree)
	return astor.to_source(new_tree)



In [6]:
source = 'f(g(1), h(2))'
print(unnest_calls(source))


temp_2 = f(

temp_0 = g(1), temp_0, 

temp_1 = h(2), temp_1), temp_2



In [10]:
import ast
import astor

def unnest_calls(code):
    class CallTransformer(ast.NodeTransformer):
        def __init__(self):
            self.temp_counter = 0

        def get_temp_name(self):
            name = f"v{self.temp_counter}"
            self.temp_counter += 1
            return name

        def visit_Call(self, node):
            # First ensure the current call's arguments are visited
            node.args = [self.visit(arg) for arg in node.args]

            # Now, replace nested calls with variables
            for i, arg in enumerate(node.args):
                if isinstance(arg, ast.Call):
                    temp_name = self.get_temp_name()
                    temp_var = ast.Name(id=temp_name, ctx=ast.Store())
                    assign = ast.Assign(targets=[temp_var], value=arg)
                    self.temp_statements.append(assign)
                    node.args[i] = ast.Name(id=temp_name, ctx=ast.Load())

            return node

        def transform(self, node):
            self.temp_statements = []
            new_node = self.visit(node)  # Visit the entire expression to handle nested calls
            self.temp_statements.append(new_node)  # Append the final expression after all modifications
            return self.temp_statements

    tree = ast.parse(code)
    transformer = CallTransformer()
    # Ensure the original statement is handled, not just the value expression
    transformed_statements = transformer.transform(tree.body[0].value)
    tree.body[0].value = ast.Tuple(elts=transformed_statements, ctx=ast.Load())
    
    return astor.to_source(tree)

# Example usage
code = "result = f(g(1), h(2))"
print(unnest_calls(code))


KeyboardInterrupt: 

In [21]:
import ast

def unnest_expression(expression):
    tree = ast.parse(expression, mode='eval')
    
    # This list will hold all our operations
    operations = []
    # This counter will help us create unique variable names
    var_counter = [0]
    
    def handle_node(node):
        if isinstance(node, ast.Num):  # For numbers
            var_name = f'v{var_counter[0]}'
            var_counter[0] += 1
            operations.append((var_name, None, node.n))
            return var_name
        elif isinstance(node, ast.Name):  # For variables
            var_name = f'v{var_counter[0]}'
            var_counter[0] += 1
            operations.append((var_name, None, node.id))
            return var_name
        elif isinstance(node, ast.Call):
            args = [handle_node(arg) for arg in node.args]
            func_name = node.func.id
            var_name = f'v{var_counter[0]}'
            var_counter[0] += 1
            operations.append((var_name, func_name, args))
            return var_name
        else:
            raise ValueError("Unsupported expression type")
    
    handle_node(tree.body)
    
    operations[-1] = (None, *operations[-1][1:])
    
    return operations

result = unnest_expression("f(g(1), h(x))")
expected = [
    ('v0', None, 1), ('v1', 'g', ['v0']), ('v2', None, 'x'), ('v3', 'h', ['v2']), (None, 'f', ['v1', 'v3'])
]
result = unnest_expression('f(g(h(x), y))')
result

[('v0', None, 'x'),
 ('v1', 'h', ['v0']),
 ('v2', None, 'y'),
 ('v3', 'g', ['v1', 'v2']),
 (None, 'f', ['v3'])]